# 1. Loading data and overview

In [ ]:
# # # Baixar o arquivo
# !wget https://www.cdc.gov/brfss/annual_data/2023/files/LLCP2023XPT.zip

# # # Descompactar o arquivo ZIP
# !unzip LLCP2023XPT.zip

# # # Verificar o nome do arquivo extraído
# import os
# os.listdir('/content/')

# # # https://www.cdc.gov/brfss/annual_data/2023/files/LLCP2023XPT.zip

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install pyreadstat
# !pip install xport

In [1]:
import pandas as pd
# import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Defina o caminho para o arquivo .XPT
file_path = 'LLCP2023.XPT '
df        = pd.read_sas(file_path, format='xport', encoding='latin1')
df.to_csv('LLCP2023.csv', index=False)
print("Arquivo convertido com sucesso para CSV!")

FileNotFoundError: [Errno 2] No such file or directory: 'LLCP2023.XPT '

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
## loading data
df = pd.read_csv('LLCP2023.csv')

In [ ]:
## overview
print("df shape -->", df.shape)
print()
df.head(5)

df shape --> (433323, 350)



,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,_RFSEAT2,_RFSEAT3,_DRNKDRV
0,1.00,1.00,3012023,3,1,2023,1100.00,2023000001,2023000001.00,1.00,...,0.00,1.00,0.00,1.00,2.00,2.00,2.00,1.00,1.00,9.00
1,1.00,1.00,1062023,1,6,2023,1100.00,2023000002,2023000002.00,1.00,...,0.00,1.00,0.00,1.00,1.00,1.00,2.00,1.00,1.00,9.00
2,1.00,1.00,3082023,3,8,2023,1100.00,2023000003,2023000003.00,1.00,...,0.00,1.00,0.00,1.00,1.00,1.00,2.00,1.00,1.00,9.00
3,1.00,1.00,3062023,3,6,2023,1100.00,2023000004,2023000004.00,1.00,...,0.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,9.00
4,1.00,1.00,1062023,1,6,2023,1100.00,2023000005,2023000005.00,1.00,...,7.00,1.00,47.00,1.00,2.00,1.00,2.00,1.00,1.00,2.00


# 2. Checking for duplicates and nulls

In [10]:
## checking duplicates
print("duplicates -->", df.duplicated().sum())
print()

## checking nulls
print(df.isnull().sum().sort_values(ascending=False))

duplicates --> 0

PAMIN13_    47837
PA3MIN_     46253
_MINAC22    46146
_MINAC12    45871
PAVIG13_    45360
            ...  
DISPCODE        0
SEQNO           0
_PSU            0
SEXVAR          0
_STATE          0
Length: 152, dtype: int64


In [11]:
## describe
print(df.describe())

              _STATE         FMONTH         IDATE         IMONTH  \
count  163435.000000  163435.000000  1.634350e+05  163435.000000   
mean       13.294961       6.725897  6.820439e+06       6.662851   
std         6.861361       3.504478  3.435974e+06       3.436362   
min         1.000000       1.000000  1.012024e+06       1.000000   
25%         8.000000       4.000000  4.062023e+06       4.000000   
50%        13.000000       7.000000  7.052023e+06       7.000000   
75%        19.000000      10.000000  1.006202e+07      10.000000   
max        24.000000      12.000000  1.231202e+07      12.000000   

                IDAY          IYEAR       DISPCODE         SEQNO  \
count  163435.000000  163435.000000  163435.000000  1.634350e+05   
mean       15.556527    2023.066087    1119.031419  2.023005e+09   
std         8.679083       0.248436      39.255018  3.140274e+03   
min         1.000000    2023.000000    1100.000000  2.023000e+09   
25%         8.000000    2023.000000    1100.000

# 3. Columns filtering

## 3.1. first columns filtering: > 30% null values

In [12]:
## removing columns with 30% of nulls
percent_missing  = df.isnull().sum()*100 / len(df)
columns_below_30 = percent_missing[percent_missing < 30]

## index to columns
index_list = columns_below_30.reset_index()['index'].tolist()

## filtering columns_below_30
df = df[index_list].copy()

## 3.2. second columns filtering: colunas que interessam para o projeto

In [13]:
## analysing each column
liana   = df.iloc[:, :49]
luiz    = df.iloc[:, 50:100]
jessica = df.iloc[:, 101:]

dict_luiz = {
             "_PACAT3" : "Physical Activity Categories",
             "_RFHYPE6": "Adults who have been told they have high blood pressure by a doctor, nurse, or other health professional",
             "_RFCHOL3": "Adults who have had their cholesterol checked and have been told by a doctor, nurse, or other health professional that it was high",
             "_MICHD"  : "Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI)",
             "_LTASTH1": "Adults who have ever been told they have asthma",
             "_AGEG5YR": "Fourteen-level age category",
             "_DRDXAR2": "Respondents who have had a doctor diagnose them as having some form of arthritis",
             "HTM4"    : "Reported height in meters",
             "WTKG3"   : "Reported weight in kilograms",
             "_BMI5CAT": "Four-level BMI categories",
             "_EDUCAG" : "Calculated education level",
             "_INCOMG1": "Calculated income level",
             "_PAINDX3": "Physical Activity Index"
            }

dict_liana = {
              'SEXVAR'  : 'Sex of Respondent',
              'GENHLTH' : 'General Health',
              'PHYSHLTH': 'Number of Days Physical Health Not Good',
              'MENTHLTH': 'Number of Days Mental Health Not Good',
              'CHECKUP1': 'Length of time since last routine checkup',
              'EXERANY2': 'Exercise in Past 30 Days',
              'EXRACT12': 'Type of Physical Activity',
              'EXERHMM1': 'Minutes or Hours Walking, Running, Jogging, or Swimming',
              'EXRACT22': 'Other Type of Physical Activity Giving Most Exercise During Past Month',
              'CVDINFR4': 'Ever Diagnosed with Heart Attack',
              'CVDCRHD4': 'Ever Diagnosed with Angina or Coronary Heart Disease',
              'CVDSTRK3': 'Ever Diagnosed with a Stroke',
              'CHCOCNC1': '(Ever told) (you had) melanoma or any other types of cancer?',
              'CHCCOPD3': 'Ever told you had C.O.P.D. emphysema or chronic bronchitis?',
              'ADDEPEV3': '(Ever told) you had a depressive disorder',
              'CHCKDNY2': 'Ever told you have kidney disease?',
              'DIABETE4': '(Ever told) you had diabetes'
             }

dict_jessica = {
              "DECIDE"  : "Because of a physical, mental, or emotional condition, do you have serious difficulty concentrating, remembering, or making decisions?",
              "DIFFALON": "Difficulty doing errands alone",
              "_PHYS14D": "3 level not good physical health status: 0 days, 1-13 days, 14-30 days",
              "_MENT14D": "3 level not good mental health status: 0 days, 1-13 days, 14-30 days",
              "MAXVO21_": "Estimated Age-Gender Specific Maximum Oxygen Consumption",
              "ACTIN13_": "Estimated Activity Intensity for First Activity",
              "STRFREQ_": "Strength Activity Frequency per Week",
              "PA3MIN_" : "Minutes of total Physical Activity per week"
              }

In [14]:
## columns to keep
columns_to_keep = list(dict_luiz.keys()) + list(dict_liana.keys()) + list(dict_jessica.keys())

## df with selected columns
df_filtered = df[columns_to_keep]

# Exibir as primeiras linhas do novo DataFrame para verificar
# print(df_filtered.head())
df_filtered.shape

(163435, 38)

# 4. refined treatment of columns

In [15]:
## weight (WTKG3)
# transforming to kg
df["WTKG3"] = df["WTKG3"]/100

# logarithm


# 5. saving filtered columns

In [16]:
## saving filtered df
df_filtered.to_csv('filtered_columns_LLCP2023.csv', index=False)

In [17]:
df_filtered.shape

(163435, 38)